# word2vec +余弦相似度

In [ ]:
#model_file = 'E:/litao/Relation_Extraction/SelfAtt_BLSTM_CH/sgns.target.word-character.char1-2.dynwin5.thr10.neg5.dim300 - 40w.txt'
model_file = 'E:/litao/Chatbots/data/word2vec/news_12g_baidubaike_20g_novel_90g_embedding_64.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True,encoding='utf-8')

In [ ]:
import gensim
import jieba
import json
import numpy as np
from scipy.linalg import norm
from pyhanlp import *




stop_words = []
f = open('E:/litao/Chatbots/data/ChineseStopWords.txt','r',encoding='ISO-8859-1')
for w in f.readlines():
    stop_words.append(w)


def sentence_vector(s):
       # words = HanLP.extractKeyword(s, 10)
        words = jieba.lcut(s)
        words = ' '.join(words).split()
        #print(words)
        v = np.zeros(64)
        clean_w = []
        for word in words:
            if word not in stop_words:
                if word in model:
                    v += model[word]
                    clean_w.append(word)
        
       # print(len(clean_w))
        if len(clean_w)>0 and v.any() >0:            
                v /= len(clean_w)
                return v
        else:
            return v
def word2vec(file_path):
    f = open(file_path,'r', encoding='utf-8')
    wv = []
    wv_dict = {}
    for  i ,line in enumerate(f.readlines()):
        line = json.loads(line)
        question = line['title']
        answer = line['answer']      
        v = sentence_vector(question)
        #if v.any() != 0:
        wv_dict[i] = (question,answer)
        wv.append(v)
    return wv,wv_dict

file_path = 'E:/litao/Chatbots/data/baike_qa/baike_qa_train.json'
wv,wv_dict = word2vec(file_path)

In [ ]:
sent1 = '松本面板可以配什么品牌的超五类模块??'
sent2 = '在线等“神通卡”的问题神通卡绑定游戏ID，能使用枪15天过了15'
v1 = sentence_vector(sent1)
v2= sentence_vector(sent2)
sim= np.dot(v1, v2) / (norm(v1) * norm(v2))
print(sim)

In [ ]:
from time import *
def vector_similarity(v1,wv,wv_dict):
    sims =  []
    sims_dict ={}
    for i,v2 in enumerate(wv):
         if v1.any() != 0 and v2.any() != 0:
                sim = np.dot(v1, v2) / (norm(v1) * norm(v2))
                #print(sim)
                #print(v2)
                sims.append(sim)
                sims_dict[sim] = i
                
                
    sims.sort(reverse=True)
    #print(sims[0])
    #print(sims_dict)
    top_n = 3
    for i in range(top_n):
        print (sims[i])
        print( wv_dict[sims_dict[sims[i]]])

begin_time = time()  

userinput1 = '下雨怎么办？'
userinput2 = '胃痛怎么办？'
userinput3 = '如何学好数学？ '
v1 = sentence_vector(userinput1)
v2 = sentence_vector(userinput2)
v3 = sentence_vector(userinput3)
print('---------')
vector_similarity(v1,wv,wv_dict)
print('---------')
vector_similarity(v2,wv,wv_dict)
print('---------')
vector_similarity(v3,wv,wv_dict)
print('---------')

end_time=time()
run_time = (end_time-begin_time)/3
print ('Word2vec相似度运行时间：',run_time)
